In [1]:
from faiss_module import make_db, make_fewshot_db
import pandas as pd

# train_df = pd.read_csv('train.csv')
# fewshot_db = make_fewshot_db(train_df)
# train_db = make_db(train_df)
# train_retriever = train_db.as_retriever(search_kwargs={'k': 1})

test_df = pd.read_csv('test.csv')
test_db = make_db(test_df)


Loading PDF files from: 9
Done. 267 chunks
Creating FAISS DB
Done.


In [22]:
querry = test_df.iloc[6]['Question']
test_retriver = test_db.as_retriever(search_kwargs={'k': 14})
print(querry)
test_retriver.invoke(querry)

from faiss_module_bw import load_and_vectorize,load_chunks_make_docdb,create_multi_query_retriever
from model_bw import setup_llm_pipeline
llm = setup_llm_pipeline()
test_retriver = create_multi_query_retriever(test_db, llm, k=3)

창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?


RuntimeError: No GPU found. A GPU is needed for quantization.

In [20]:
test_retriver = test_db.as_retriever(search_kwargs={'search_tpye':'ngram','k': 14})
print(querry)
test_retriver.invoke(querry)

우리나라에서는 언제부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있는가?


[Document(metadata={'source': './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf', 'page': 1}, page_content='- 국제적으로 정부의 채무지속가능성분석(DSA)4)을 위한 기초자료로 활용되는 등 주요하게 인식되는 추세와 \n비교해 우리나라에서는 아직까지 체계적으로 다뤄지지 않고 있음\nFOCUS   우발부채 관련 주요 쟁점은?\n‣ 우발부채 개념 및 용어 사용의 혼란, 우발부채 분류기준 재검토, 새로운 분류기준 정립\n    - 발생주의 회계기준을 적용한 국가결산보고서가 10년간 작성되었으나 우발부채의 인식과 공시는 크게 달라지지 \n않음(현재 주석에서 ‘우발사항 및 약정사항’으로 보고)\n    - 우리나라는 국가회계기준, 지방회계기준, 한국채택국제회계기준(K–IFRS) 등에서 우발부채를 광의적 개념에\n서만 정의하고 있어 용어 사용 혼란 및 체계적 관리 한계(본고 4페이지 참고)\n    - SNA, PSDS, IPSAS 등 다양한 국제기준 통계가 GFS체계로 조화를 모색하는 추세임을 감안할 때,'),
 Document(metadata={'source': './test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》.pdf', 'page': 1}, page_content='02\nFIS    ISSUE & FOCUS                    들어가며\nISSUE   왜 우발부채에 주목하는가?\n‣ 국제기준 재정통계 산출에 발생주의(accrual basis)1) 회계기준이 적용되면서 미래의 다양한 의제의무\n(constructive obligation)2)를 포괄하는 우발부채 관리에 대한 중요성 인식\n    - 기업회계에서는 우발부채 규모가 그리 크지 않아 중요한 요소가 아니지만, 공공부문에서는 재정건전성, 재정\n위기 관리 등 다양한 관점에서 중요하게 다뤄져야 할 필요\n    - GFSM 2001에서 광의적 개념으로 정의하였던 것에 비해,